In [1]:

import torch
import torch.nn as nn
import mmap
import random
import pickle
from torchsummary import summary

from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


batch_size = 128
block_size = 32
max_iters = 100
learning_rate = 3e-4
eval_iters = 10
n_embd=768
n_head = 8
n_layer = 8
dropout = 0.2


cuda


In [2]:
chars = ""
with open ('vocab.txt','r',encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)


['\x00', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x88', '\x8c', '\x8d', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9c', '\x9d', '\x9f', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã

In [3]:

if '<unk>' not in chars:
    chars.append('<unk>')

# Create mappings
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# ID for unknown character
UNK_TOKEN = string_to_int['<unk>']

# Encoding and decoding functions
encode = lambda s: [string_to_int.get(c, UNK_TOKEN) for c in s]
decode = lambda l: ''.join([int_to_string.get(i, '<unk>') for i in l])




x= encode('hello!')

print(x)
print(decode(x))

[74, 71, 78, 78, 81, 3]
hello!


In [4]:
def get_random_chunk(split):
    filename = "train_split.txt" if split == 'train' else "val_split.txt"
    with open(filename,'rb') as f:
        with mmap.mmap(f.fileno(),0,access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos = random.randint(0,(file_size)-block_size*batch_size)

            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            decoded_block = block.decode('utf-8',errors='ignore').replace('\r','')
            data = torch.tensor(encode(decoded_block),dtype = torch.long)
    return data



def get_batch(split):
    
    data = get_random_chunk(split)
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[70, 75, 72,  ...,  2, 67, 80],
        [91,  2, 86,  ...,  2, 70, 71],
        [23, 21,  0,  ...,  0,  0,  0],
        ...,
        [ 2, 78, 67,  ..., 67, 80, 87],
        [16,  1,  1,  ..., 82, 67, 75],
        [ 0,  0,  0,  ...,  0,  0,  0]], device='cuda:0')
targets:
tensor([[75, 72, 91,  ..., 67, 80, 91],
        [ 2, 86, 74,  ..., 70, 71, 72],
        [21,  0, 18,  ...,  0,  0,  0],
        ...,
        [78, 67, 85,  ..., 80, 87, 69],
        [ 1,  1, 35,  ..., 67, 75, 84],
        [ 0,  0,  0,  ...,  0,  0,  0]], device='cuda:0')


In [5]:
class Head(nn.Module):
    

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
       
        B,T,C = x.shape
        k = self.key(x)  
        q = self.query(x) 
      
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
      
        v = self.value(x) 
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):
  

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
   

    def __init__(self, n_embd, n_head):
       
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
     
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            index_cond = index[:, -block_size:]
            
            logits, loss = self.forward(index_cond)
           
            logits = logits[:, -1, :] 
           
            probs = F.softmax(logits, dim=-1) 
            
            index_next = torch.multinomial(probs, num_samples=1) 
           
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)

m = model.to(device)




In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model (X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)



for iter in range(max_iters):
    print(iter)
   
    if iter % eval_iters == 0:
        
        losses = estimate_loss()
        print(f"step: {iter}, train_loss: {losses['train']:.3f} , val_loss: {losses['val']:.3f}")

    xb,yb = get_batch('train')
    
    
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    

    

print(loss.item())


with open('model_01.pkl','wb') as f:
    pickle.dump(model,f)

print("Model is saved.")



0
step: 0, train_loss: 8.585 , val_loss: 8.584
1
2
3
4
5
6
7
8
9
10
step: 10, train_loss: 3.394 , val_loss: 3.123
11
12
13
14
15
16
17
18
19
20
step: 20, train_loss: 3.034 , val_loss: 2.907
21
22
23
24
25
26
27
28
29
30
step: 30, train_loss: 2.993 , val_loss: 2.594
31
32
33
34
35
36
37
38
39
40
step: 40, train_loss: 2.868 , val_loss: 2.801
41
42
43
44
45
46
47
48
49
50
step: 50, train_loss: 2.776 , val_loss: 2.948
51
52
53
54
55
56
57
58
59
60
step: 60, train_loss: 2.852 , val_loss: 3.042
61
62
63
64
65
66
67
68
69
70
step: 70, train_loss: 2.877 , val_loss: 2.835
71
72
73
74
75
76
77
78
79
80
step: 80, train_loss: 2.769 , val_loss: 2.852
81
82
83
84
85
86
87
88
89
90
step: 90, train_loss: 2.670 , val_loss: 2.882
91
92
93
94
95
96
97
98
99
2.421624183654785
Model is saved.


In [8]:
# Save entire model (not just weights)
torch.save(model, 'gpt_model_full.pt')

# OR better: Save only the model weights (recommended)
torch.save(model.state_dict(), 'gpt_model_weights.pth')

print("Model saved successfully.")


Model saved successfully.


In [9]:
"""print("loading model parameters...")
with open('model_01.pkl','rb') as f:
    model = pickle.load(f)
print("Loaded Successfully")"""

loading model parameters...
Loaded Successfully


In [10]:


context = torch.zeros((10,10) , dtype= torch.long,device = device)
generated_chars = decode(m.generate(context,max_new_tokens=150)[0].tolist())
#print(m.generate(context,max_new_tokens=150)[0].tolist())
print(generated_chars)


                                                                                                                                                                


In [11]:
total_params = sum(p.numel() for p in model.parameters())
def format_params(n):
    if n >= 1_000_000_000:
        return f"{n/1_000_000_000:.2f}B"
    elif n >= 1_000_000:
        return f"{n/1_000_000:.2f}M"
    else:
        return f"{n:,}"

print(f"Total parameters: {format_params(total_params)}")

Total parameters: 65.19M


In [12]:
print(f"Embedding table: {model.token_embedding_table.weight.shape}")
print(f"Position embedding: {model.position_embedding_table.weight.shape}")
print(f"Number of blocks: {len(model.blocks)}")
print(f"Block[0] attention head size: {model.blocks[0].sa.heads[0].key.out_features}")
print(f"Final linear layer: {model.lm_head.weight.shape}")

Embedding table: torch.Size([5520, 768])
Position embedding: torch.Size([32, 768])
Number of blocks: 8
Block[0] attention head size: 96
Final linear layer: torch.Size([5520, 768])
